In [1]:
# =========================================
# LangChain RAG: PDF Load + FAISS Similarity Search
# Colab Notebook Ready
# =========================================

# Install necessary libraries
!pip install --quiet langchain unstructured PyPDF2 faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.


In [2]:
!pip install -U langchain-community

  Using cached langchain_core-1.0.4-py3-none-any.whl.metadata (3.5 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_core-1.0.4-py3-none-any.whl (471 kB)
Using cached langchain_text_splitters-1.0.0-py3-none-any.whl (33 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
langchain 0.3.27

In [3]:
# Import libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings


In [4]:
# 1. Load PDF
# =========================================
pdf_path = "/content/sample_data/Publication.pdf"  # <-- Upload your PDF here
loader = PyPDFLoader(pdf_path)
documents = loader.load()


In [5]:
print(f"Loaded {len(documents)} document(s) from PDF.")

Loaded 20 document(s) from PDF.


In [6]:
# =========================================
# 2. Chunk the PDF into smaller pieces
# =========================================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # Size of each chunk
    chunk_overlap=200  # Overlap between chunks
)
docs = text_splitter.split_documents(documents)

print(f"Split into {len(docs)} chunks.")

Split into 116 chunks.


In [13]:
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/bert-base-nli-mean-tokens")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# 4. Build FAISS index
# =========================================
faiss_index = FAISS.from_documents(docs, embedding_model)
print("FAISS index created successfully!")


FAISS index created successfully!


In [15]:
# 5. Similarity search
# =========================================
query = "what is the main model in this document?"
results = faiss_index.similarity_search(query, k=3)  # top 3 results


In [16]:
print(f"Top {len(results)} results:")
for i, res in enumerate(results):
    print(f"\nResult {i+1}:\n{res.page_content}\n")


Top 3 results:

Result 1:
the process of generating a fused feature vector, Ofused ,
by combining individual feature vectors, OMentalBERT and
OMelBERT from their respective branches.
OMentalBERT =concat(AP1, ..,APn), O ∈ RF1,..,Fn for n=3
(15)
OMelBERT =concat(AP1, ..,APn), O ∈ RF1,..,Fn for n=3
(16)
Ofused =concat(OMentalBERT, OMelBERT), Ofused ∈R768
(17)
F. MENTAL ILLNESS CLASSIFICATION
The concatenated feature vector is fed to the fully connected
layer, also known as the dense layer, for classification. The
softmax function is employed as the activation function for
this classification process, specifically to classify into four
mental illness classes: depression, anxiety, BPD, and PTSD.
The final layer produces a vector of scores, where each
element represents the model’s confidence that the input is
Algorithm 1 Hybrid Transformer Architecture With Cas-
caded CNNs
1: Input ← Text data D, labels Y , MentalBERT model
MMental, MelBERT model MMel, CNN layers L,
kernel sizes K, fully co